# Jointformer Training

This notebook shows how to train Jointformer. 

In [1]:
# Imports

import os

from jointformer.configs.dataset import DatasetConfig
from jointformer.configs.tokenizer import TokenizerConfig
from jointformer.configs.model import ModelConfig
from jointformer.configs.trainer import TrainerConfig

from jointformer.utils.datasets.auto import AutoDataset
from jointformer.utils.tokenizers.auto import AutoTokenizer
from jointformer.models.auto import AutoModel
from jointformer.trainers.trainer import Trainer

%load_ext autoreload
%autoreload 2

/home/adamizdebski/miniconda3/envs/jointformer-experiments/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
2024-10-29 16:09:01.347627: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 16:09:01.375349: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 16:09:01.375374: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cu

In [2]:
# Set working directory of the project

REPOSITORY_DIR = '/home/adamizdebski/projects/jointformer'
os.chdir(REPOSITORY_DIR)

In [3]:
# Configs

DATA_DIR = '/home/adamizdebski/files/data'
OUTPUT_DIR = '/home/adamizdebski/files/jointformer/results/finetune'

PATH_TO_DATASET_CONFIG   = '/home/adamizdebski/projects/jointformer/configs/datasets/molecule_net/scaffold/lipo'
PATH_TO_TOKENIZER_CONFIG = '/home/adamizdebski/projects/jointformer/configs/tokenizers/gpt_tokenizer'
PATH_TO_MODEL_CONFIG = '/home/adamizdebski/projects/jointformer/configs/models/fancy_model_prediction'
PATH_TO_TRAINER_CONFIG = '/home/adamizdebski/projects/jointformer/configs/trainers/test'

In [4]:
# Test Datsaset

dataset_config = DatasetConfig.from_config_file(PATH_TO_DATASET_CONFIG)
tokenizer_config = TokenizerConfig.from_config_file(PATH_TO_TOKENIZER_CONFIG)

train_dataset = AutoDataset.from_config(dataset_config, data_dir=DATA_DIR, split='train')
val_dataset = AutoDataset.from_config(dataset_config, data_dir=DATA_DIR, split='val')

tokenizer = AutoTokenizer.from_config(tokenizer_config)

In [5]:
# Init Jointformer

model_config = ModelConfig.from_config_file(PATH_TO_MODEL_CONFIG)
model = AutoModel.from_config(model_config, downstream_task=dataset_config.task_type, num_tasks=dataset_config.num_tasks, hidden_dim=256)
# model.load_pretrained('ckpt.pt')

number of parameters: 6.55M


In [7]:
model

FancyModelForDownstreamPrediction(
  (transformer): ModuleDict(
    (wte): Embedding(593, 256)
    (type_emb): Embedding(2, 256)
    (wpe): Embedding(128, 256)
    (prop_emb): Linear(in_features=200, out_features=256, bias=True)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-7): 8 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=256, out_features=768, bias=False)
          (c_proj): Linear(in_features=256, out_features=256, bias=False)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=256, out_features=1024, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1024, out_features=256, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_

In [8]:

trainer_config = TrainerConfig.from_config_file(PATH_TO_TRAINER_CONFIG)
trainer = Trainer(
    config=trainer_config,
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    tokenizer=tokenizer
    )

num decayed parameter tensors: 40, with 6,644,737 parameters
num non-decayed parameter tensors: 19, with 4,864 parameters
using fused AdamW: True


In [24]:
trainer.train()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/adamizdebski/miniconda3/envs/jointformer-experiments/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_480/4032920361.py", line 1, in <module>
    trainer.train()
  File "/home/adamizdebski/projects/jointformer/jointformer/trainers/trainer.py", line 447, in train
    self.evaluate()
  File "/home/adamizdebski/projects/jointformer/jointformer/trainers/trainer.py", line 371, in evaluate
    losses = self.estimate_loss()
  File "/home/adamizdebski/miniconda3/envs/jointformer-experiments/lib/python3.9/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/adamizdebski/projects/jointformer/jointformer/trainers/trainer.py", line 316, in estimate_loss
    outputs = self.model.get_loss(**inputs)
  File "/home/adamizdebski/projects/jointformer/jointformer/models/fancy_mod

In [10]:
trainer._init_data_loaders()

batch = next(iter(trainer.train_loader))
batch.to('cuda')
model.to('cuda')
outputs = model(**batch)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x1 and 200x256)

In [42]:
outputs['logits_prediction'].flatten().shape

torch.Size([2])

In [40]:
outputs['embeddings'][:, -1, :]

tensor([[ 1.6961e-01,  2.6618e-01, -2.2679e-01, -6.1008e-01,  9.9106e-03,
          4.1189e-01,  1.4348e-01,  1.1176e-01,  6.5966e-01, -1.7233e+00,
          1.2009e+00, -3.4084e-01,  1.7629e+00, -1.9667e-01, -1.2688e+00,
          9.9074e-02, -8.8256e-01,  3.0045e-01,  1.8028e-01,  3.0680e-01,
         -5.4947e-01,  1.6914e+00,  3.4691e-01, -6.9734e-01, -1.9162e+00,
         -9.3154e-01, -9.1630e-01, -8.9729e-01, -1.7263e+00, -1.6298e+00,
          7.5439e-01, -5.4682e-02, -6.0041e-01, -1.3833e+00,  8.2129e-01,
         -8.0695e-01, -9.0263e-02,  4.9857e-01,  2.0863e+00, -7.5034e-01,
         -7.6549e-01, -7.6526e-01, -9.7997e-01,  3.2173e-01,  6.4486e-01,
          1.2544e+00,  7.0545e-01,  6.0114e-01,  6.6038e-01, -4.0036e-01,
          1.3329e+00, -6.0918e-01,  1.2346e+00, -1.1820e+00, -1.5737e-01,
         -3.0718e-01,  5.4611e-01,  8.7681e-01,  1.4073e+00, -1.7978e+00,
         -5.6693e-01,  1.0519e+00,  1.4003e+00,  1.6440e+00, -2.4737e-01,
         -8.5858e-02, -9.4875e-01, -7.

In [41]:
batch["properties"].flatten()

tensor([ 1.1041, -1.7448], device='cuda:0')

In [11]:
import torch

In [14]:
x = torch.rand(16, 200)
layer = torch.nn.Linear(1, 32)

In [17]:
embed = layer(x.unsqueeze(-1))

In [18]:
embed.size()

torch.Size([16, 200, 32])

In [8]:
key = 'splitter'

dataset_config[key]

'scaffold'

In [11]:
for key, value in dataset_config.__dict__.items():
    print(key, value)
    print(isinstance(key, str))

path_to_train_data None
True
path_to_train_properties None
True
path_to_val_data None
True
path_to_val_properties None
True
path_to_test_data None
True
path_to_test_properties None
True
dataset_name molecule_net
True
target_label lipo
True
data_filepath None
True
tatget_filepath None
True
validate False
True
standardize False
True
num_samples None
True
split val
True
splitter scaffold
True
transform None
True
target_transform None
True
task_type regression
True
metric rmse
True
num_tasks 1
True


In [13]:
model_config["model_name"]

'FancyModelForDownstreamPrediction'

In [15]:
"model_name" in model_config.__dict__.keys()

True